In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [0]:
from google.colab import files
uploaded = files.upload()

Saving sonar.csv to sonar.csv


In [3]:
# load dataset
dataframe = pd.read_csv("sonar.csv", header=None)
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

In [4]:
# baseline model
def create_baseline():
    model = Sequential()
    model.add(Dense(60, activation='relu')) 
    model.add(Dense(1, activation='sigmoid'))
  
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [5]:
encoded_Y = LabelEncoder()

encoded_Y = encoded_Y.fit(Y).transform(Y)

## Step 2. Baseline Neural Network Model Performance

In [6]:
#evaluate model with standardized dataset

estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 84.59% (6.45%)


## Step 3. Re-Run The Baseline Model With Data Preparation

In [8]:
#evaluate baseline model with standardized dataset

np.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 84.09% (5.82%)


## Step 4. Tuning Layers and Number of Neurons in The Model

#### 4.1. Evaluate a Smaller Network

In [0]:
# smaller model

def create_smaller():
    small_model = Sequential()
    small_model.add(Dense(30, activation='relu')) 
    small_model.add(Dense(1, activation='sigmoid'))
  
    small_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return small_model
  
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Smaller model results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

#### 4.2. Evaluate a Larger Network

In [12]:
# larger model

def create_larger():
    large_model = Sequential()
    large_model.add(Dense(60, activation='relu'))
    large_model.add(Dense(30, activation='relu')) 
    large_model.add(Dense(1, activation='sigmoid'))
  
    large_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return large_model
  
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs=100, batch_size=5, verbose=0))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger model result: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

SyntaxError: invalid syntax (<ipython-input-12-e71effdd63de>, line 15)

## Step 5. Really Scaling up: developing a model that overfits

In [0]:
# overfit model
def create_overfit():
    overfit_model = Sequential()
    overfit_model.add(Dense(120, activation='relu'))
    overfit_model.add(Dense(120, activation='relu'))
    overfit_model.add(Dense(60, activation='relu'))
    overfit_model.add(Dense(60, activation='relu')) 
    overfit_model.add(Dense(1, activation='sigmoid'))
  
    overfit_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return overfit_model
  
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_overfit, epochs=1000, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=25, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Overfit result: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

## Step 6: Tuning the Model with weight regularization

In [13]:
from keras import regularizers

# Two hidden layers model
def tuning_one():
    tuning1_model = Sequential()
    tuning1_model.add(Dense(60, kernel_regularizer=regularizers.l2(0.001), activation='relu'))
    tuning1_model.add(Dense(60, kernel_regularizer=regularizers.l2(0.001), activation='relu')) 
    tuning1_model.add(Dense(1, activation='sigmoid'))
  
    tuning1_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return tuning1_model
  
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=tuning_one, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 86.49% (5.68%)


In [0]:
# Two hidden layers with changes in nodes model
def tuning_two():
    tuning2_model = Sequential()
    tuning2_model.add(Dense(120, kernel_regularizer=regularizers.l2(0.001), activation='relu'))
    tuning2_model.add(Dense(60, kernel_regularizer=regularizers.l2(0.001), activation='relu')) 
    tuning2_model.add(Dense(1, activation='sigmoid'))
  
    tuning2_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return tuning2_model
  
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=tuning_two, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 84.56% (6.12%)


In [0]:
# Changes in optimizer
def tuning_three():
    tuning3_model = Sequential()
    tuning3_model.add(Dense(60, kernel_regularizer=regularizers.l2(0.001), activation='relu'))
    tuning3_model.add(Dense(60, kernel_regularizer=regularizers.l2(0.001), activation='relu')) 
    tuning3_model.add(Dense(1, activation='sigmoid'))
  
    tuning3_model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    return tuning3_model
  
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=tuning_three, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [0]:
# Changes in epochs
def tuning_four():
    tuning4_model = Sequential()
    tuning4_model.add(Dense(60, kernel_regularizer=regularizers.l2(0.001), activation='relu'))
    tuning4_model.add(Dense(60, kernel_regularizer=regularizers.l2(0.001), activation='relu')) 
    tuning4_model.add(Dense(1, activation='sigmoid'))
  
    tuning4_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return tuning4_model
  
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=tuning_four, epochs=500, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

## Step 7: Rewriting the code using the Keras Functional API

In [0]:
from keras.layers import Input, Dense
from keras.models import Model

def api_model():
    inputs = Input(shape=(60,))

    x = Dense(60, activation='relu')(inputs)
    x = Dense(60, activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)

    APImodel = Model(inputs=inputs, outputs=predictions)
    APImodel.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
    return APImodel

In [0]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=api_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 83.61% (6.20%)


## Step 8: Rewriting the code by doing Model Subclassing

In [0]:
import keras

class MyModel(keras.Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.dense1 = Dense(60, activation='relu')
    self.dense2 = Dense(60, activation='relu')
    self.dense3 = Dense(1, activation='sigmoid')

  def call(self, inputs):
    x = self.dense1(inputs)
    x = self.dense2(x)
    return self.dense3(x)

def modelSubClassing():
  subclass_model = MyModel()
  subclass_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return subclass_model

In [0]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=modelSubClassing, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 83.09% (4.69%)


## Step 9: Rewriting the code without using scikit-learn

In [18]:
from sklearn.utils import shuffle

shuffled_X, shuffled_Y = shuffle(X, encoded_Y, random_state=0)

In [19]:
k = 4

num_val_samples=len(X)//k
num_epochs = 50

In [20]:
for i in range(k):
    print('Processing fold number', i+1)

    val_data = shuffled_X[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = shuffled_Y[i * num_val_samples: (i + 1) * num_val_samples]

    partial_train_data = np.concatenate([shuffled_X[:i * num_val_samples], shuffled_X[(i + 1) * num_val_samples:]], axis=0)

    partial_train_targets = np.concatenate([shuffled_Y[:i * num_val_samples],shuffled_Y[(i + 1) * num_val_samples:]],axis=0)

    model = create_baseline()

    results = model.fit(partial_train_data, partial_train_targets, epochs=num_epochs, batch_size=5, verbose=0, validation_data=(val_data, val_targets))
    loss, acc = model.evaluate(val_data, val_targets, verbose=0)
    historyy = results.history
    #print(historyy['acc'])
    print(acc)

Processing fold number 1
0.8461538553237915
Processing fold number 2
0.7692307829856873
Processing fold number 3
0.7692307829856873
Processing fold number 4
0.807692289352417


In [21]:
model.save("Sonar2.h5")